In [2]:
from pathlib import Path
from nvi_etl import make_engine_for
from nvi_etl.destinations import CONTEXT_VALUES_TABLE, SURVEY_VALUES_TABLE
from sqlalchemy import text
import pandas as pd

In [3]:
data_engine = make_engine_for("data")

In [3]:
q = text(f"""
SELECT *
FROM nvi.{SURVEY_VALUES_TABLE}
""")

secondary_df = pd.read_sql(q, data_engine)

primary_path = Path.cwd() / "primary_survey" / "2024" / "output" / "primary_output.csv"
primary_df = pd.read_csv(primary_path)

In [4]:
result = (
    pd.concat([secondary_df, primary_df])
    .astype({"location_id": pd.Int64Dtype()})
    .drop_duplicates(subset=[
        "indicator_id", 
        "location_id", 
        "survey_id", 
        "survey_question_id",
        "survey_question_option_id",
        "year",
    ])
)

C:\Users\mike\AppData\Local\Temp\ipykernel_29024\2412312132.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([secondary_df, primary_df])


In [5]:
ready = (
    result
    .drop(["Unnamed: 0"], axis=1)
    .rename(columns={
        'indicator_id': 'indicator',
        'location_id': 'location',
        'survey_id': 'survey',
        'survey_question_id': 'survey_question',
        'survey_question_option_id': 'survey_question_option',
    })
    .astype({
        'indicator': pd.Int64Dtype(),
        'location': pd.Int64Dtype(),
        'survey': pd.Int64Dtype(),
        'survey_question': pd.Int64Dtype(),
        'survey_question_option': pd.Int64Dtype(),
        'count': pd.Int64Dtype(),
        'universe': pd.Int64Dtype(),
        'rate_per': pd.Int64Dtype(),
        'dollars': float
    })
    .fillna({"survey": 1})
    .assign(id=lambda df: range(len(df)), value_type=1)
)

In [6]:
len(ready)

5070

In [7]:
n = 500
chunks = [ready[i:i+n] for i in range(0, ready.shape[0], n)]

for i, chunk in enumerate(chunks):
    chunk.to_csv(f"nvi_20250401_{i+1}.csv", index=False)

In [161]:
from nvi_etl.geo_reference import pin_location

In [ ]:
file = pd.read_csv(Path.cwd() / "secondary_msc" / "2024" / "input" / "msc_wide_from_queries.csv")

wide = (
    file
    .rename(columns={
        'total_crash': 'count_auto_crash',
        'auto_crash_per_10000': 'rate_auto_crash',
        'total_ped_crash': 'count_ped_crash',
        'ped_crash_per_10000': 'rate_ped_crash',
        'num_cdos': 'cdo_count',
        'pct_cdo_coverage': 'percentage_cdo_coverage',
        'total_violent_crimes': 'count_violent_crimes',
        'crime_rate_per_10000': 'rate_violent_crimes',
    })
    .assign(
        per_auto_crash=10_000,
        per_ped_crash=10_000,
        per_violent_crimes=10_000,
    )
)[[
    "geo_type", 
    "geography", 

    "count_auto_crash", 
    "rate_auto_crash", 
    "per_auto_crash",

    "count_ped_crash",
    "rate_ped_crash",
    "per_ped_crash",

    "percentage_cdo_coverage",

    "count_violent_crimes",
    "rate_violent_crimes",
    "per_violent_crimes",
]]

(
    pd.wide_to_long(
        wide,
        stubnames=["count", "universe",  "percentage", "rate", "per", "dollars", "index"],
        i=["geo_type", "geography",],
        j="indicator_name",
        sep="_",
        suffix=".*"
    )
    .reset_index()
    .rename(columns={"per": "rate_per"})
    .assign(
        location=lambda df: df.apply(pin_location, axis=1)
    )
    .astype({
        "count": pd.Int64Dtype(),
        "universe": pd.Int64Dtype(),
        "percentage": pd.Float64Dtype(),
        "rate_per": pd.Int64Dtype(),
        "rate": pd.Float64Dtype(),
        "dollars": pd.Float64Dtype(),
        "index": pd.Float64Dtype(),
    })
    .sort_values("location")
)

,geo_type,geography,indicator_name,count,universe,percentage,rate,rate_per,dollars,index,location
31,citywide,Detroit,cdo_coverage,<NA>,<NA>,74.201704,<NA>,<NA>,<NA>,<NA>,1
30,citywide,Detroit,violent_crimes,2659,<NA>,<NA>,41.765885,10000,<NA>,<NA>,1
29,citywide,Detroit,ped_crash,567,<NA>,<NA>,8.906076,10000,<NA>,<NA>,1
28,citywide,Detroit,auto_crash,23650,<NA>,<NA>,371.479194,10000,<NA>,<NA>,1
0,district,1,auto_crash,3010,<NA>,<NA>,316.103421,10000,<NA>,<NA>,2
...,...,...,...,...,...,...,...,...,...,...,...
115,zone,7b,cdo_coverage,<NA>,<NA>,64.529702,<NA>,<NA>,<NA>,<NA>,30
116,zone,7c,auto_crash,1207,<NA>,<NA>,373.533872,10000,<NA>,<NA>,31
117,zone,7c,ped_crash,34,<NA>,<NA>,10.522081,10000,<NA>,<NA>,31
118,zone,7c,violent_crimes,166,<NA>,<NA>,51.372513,10000,<NA>,<NA>,31


In [16]:
secondary_df[[
    "indicator_id",
    "location_id",
    "survey_id",
    "survey_question_id",
    "survey_question_option_id",
    "year",
]].value_counts(dropna=False).sort_values()

indicator_id  location_id  survey_id  survey_question_id  survey_question_option_id  year
4             1            NaN        NaN                 NaN                        2023    1
              2            NaN        NaN                 NaN                        2023    1
              3            NaN        NaN                 NaN                        2023    1
              4            NaN        NaN                 NaN                        2023    1
              5            NaN        NaN                 NaN                        2023    1
                                                                                            ..
              29           NaN        NaN                 NaN                        2023    1
              30           NaN        NaN                 NaN                        2023    1
              31           NaN        NaN                 NaN                        2023    1
7             1            NaN        NaN              

In [ ]:
"""
SELECT *
FROM (
	SELECT year,
    indicator_id,
    location_id,
    survey_id,
    survey_question_id,
    survey_question_option_id,
    rank() over (
        partition by year,
        indicator_id,
        location_id,
        survey_id,
        survey_question_id,
        survey_question_option_id order by id
    )
	from value
) ranked
where rank > 1;
"""

In [10]:
q = text(
    f"""
    SELECT *
    FROM nvi.{CONTEXT_VALUES_TABLE}
    WHERE indicator_id = 13;
    """
)

df = pd.read_sql(q, data_engine)

In [11]:
df.to_csv("context_indicators_update_20250409_3.csv", index=False)